# [5주차 기본과제] 뉴스 기사 분류를 Gemma로 구현하기


In [88]:
import os
from dotenv import load_dotenv

load_dotenv()
HF_TOKEN = os.environ.get("HF_TOKEN")

from huggingface_hub import login

login(HF_TOKEN)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/obov/.cache/huggingface/token
Login successful


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [89]:
from datasets import load_dataset


news_ds = load_dataset("fancyzhx/ag_news")
news_ds

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [90]:
world_ds = news_ds["train"].filter(lambda data: data["label"] == 0)
world_ds["text"][:3]

['Venezuelans Vote Early in Referendum on Chavez Rule (Reuters) Reuters - Venezuelans turned out early\\and in large numbers on Sunday to vote in a historic referendum\\that will either remove left-wing President Hugo Chavez from\\office or give him a new mandate to govern for the next two\\years.',
 'S.Koreans Clash with Police on Iraq Troop Dispatch (Reuters) Reuters - South Korean police used water cannon in\\central Seoul Sunday to disperse at least 7,000 protesters\\urging the government to reverse a controversial decision to\\send more troops to Iraq.',
 "Palestinians in Israeli Jails Start Hunger Strike (Reuters) Reuters - Thousands of Palestinian\\prisoners in Israeli jails began a hunger strike for better\\conditions Sunday, but Israel's security minister said he\\didn't care if they starved to death."]

In [91]:
sports_ds = news_ds["train"].filter(lambda data: data["label"] == 1)
sports_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 30000
})

In [92]:
business_ds = news_ds["train"].filter(lambda data: data["label"] == 2)
business_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 30000
})

In [93]:
sci_tech_ds = news_ds["train"].filter(lambda data: data["label"] == 3)
sci_tech_ds

Dataset({
    features: ['text', 'label'],
    num_rows: 30000
})

In [94]:
news_ds["train"].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}

In [95]:
label_map = news_ds["train"].features["label"].names

In [97]:
news_ds["train"].info

DatasetInfo(description='', citation='', homepage='', license='', features={'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['World', 'Sports', 'Business', 'Sci/Tech'], id=None)}, post_processed=None, supervised_keys=None, builder_name='parquet', dataset_name='ag_news', config_name='default', version=0.0.0, splits={'train': SplitInfo(name='train', num_bytes=29832303, num_examples=120000, shard_lengths=None, dataset_name='ag_news'), 'test': SplitInfo(name='test', num_bytes=1880424, num_examples=7600, shard_lengths=None, dataset_name='ag_news')}, download_checksums={'hf://datasets/fancyzhx/ag_news@eb185aade064a813bc0b7f42de02595523103ca4/data/train-00000-of-00001.parquet': {'num_bytes': 18585438, 'checksum': None}, 'hf://datasets/fancyzhx/ag_news@eb185aade064a813bc0b7f42de02595523103ca4/data/test-00000-of-00001.parquet': {'num_bytes': 1234829, 'checksum': None}}, download_size=19820267, post_processing_size=None, dataset_size=31712727, size_in_bytes=51532994)

In [98]:
from pprint import pprint

pprint(news_ds["train"].info)

DatasetInfo(description='',
            citation='',
            homepage='',
            license='',
            features={'label': ClassLabel(names=['World',
                                                 'Sports',
                                                 'Business',
                                                 'Sci/Tech'],
                                          id=None),
                      'text': Value(dtype='string', id=None)},
            post_processed=None,
            supervised_keys=None,
            builder_name='parquet',
            dataset_name='ag_news',
            config_name='default',
            version=0.0.0,
            splits={'test': SplitInfo(name='test',
                                      num_bytes=1880424,
                                      num_examples=7600,
                                      shard_lengths=None,
                                      dataset_name='ag_news'),
                    'train': SplitInfo(name='train',
   

In [99]:
tokenizer("World", return_tensors="pt")
tokenizer("World", return_tensors="pt")["input_ids"].shape[-1]

2

In [100]:
label_sentences = [
    "Batman: This is mainly about the world, the one we’re sworn to protect.",
    "Joker: Why so serious? Clearly related to Sports news!",
    "Penguin: It’s all about Business, and trust me, I know how to play that game.",
    "Bane: This article mainly covers Sci/Tech.",
]

task_description = """
Task Description:
The scene unfolds with four iconic characters from the Batman universe seated around a table, each reading the same article. Batman, ever composed and logical, is the first to speak, delivering a clear and concise assessment of the piece. The Joker follows, making a mocking and unpredictable remark, turning the topic on its head. Two-Face, as always, relies on the flip of a coin to determine his take. Finally, Bane, with his focus on power and control, offers his perspective, emphasizing the article’s deeper implications.

Your task is simple: “Who explained the article the best?”.

Now, it’s time to choose—who described it best?

ARTICLE:"""

In [101]:
import torch


def zero_shot_classification(
    text, task_description, labels
):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to(
        "mps"
    )  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.

    probs = []

    for label in labels:
        # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("mps")
        n_label_tokens = label_ids["input_ids"].shape[-1] - 1
        # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            "input_ids": torch.concatenate(
                [text_ids["input_ids"], label_ids["input_ids"][:, 1:]], axis=-1
            ),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            "attention_mask": torch.concatenate(
                [text_ids["attention_mask"], label_ids["attention_mask"][:, 1:]],
                axis=-1,
            ),
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids["input_ids"].shape[-1]
        for i in range(
            n_label_tokens, 0, -1
        ):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids["input_ids"][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.mps.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

In [102]:
news_ds["train"][1]["text"]

'Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\\which has a reputation for making well-timed and occasionally\\controversial plays in the defense industry, has quietly placed\\its bets on another part of the market.'

In [103]:
news_ds["train"][1]["label"]

2

In [104]:
text_label = news_ds["train"][179]

probs = zero_shot_classification(
    text_label["text"],
    task_description,
    label_sentences,
)
print(probs)
import numpy as np

print(text_label["label"])
np.argmax(np.array(probs)).item()

[99.28125, 70.71484375, 158.25, 48.431640625]
3


2

In [86]:
def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)


tokenized_news_ds = news_ds.map(preprocess_function, batched=True)
tokenized_news_ds

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7600
    })
})

In [106]:
import numpy as np
from tqdm import tqdm
from colorama import Fore, Style, Back
import torch.nn.functional as F
import humanize
import math
import random


def calculate_nll(probs_softmax, label):
    return -torch.log(probs_softmax[label])


def get_confidence(probs_softmax):
    top_two_probs, _ = torch.topk(probs_softmax, 2)
    # 가장 높은 확률을 두 번째로 높은 확률로 나누어 비율을 계산
    return math.log10(top_two_probs[0] / top_two_probs[1])


total = 50
n_corrects = 0

# 0부터 7600 사이에서 겹치지 않게 50개의 숫자 뽑기
random_numbers = random.sample(range(7601), total)

for i in tqdm(range(total)):

    print(f"{Fore.YELLOW}----------------------------------{Fore.RESET}")
    text = tokenized_news_ds["test"][random_numbers[i]]["text"]
    label = tokenized_news_ds["test"][random_numbers[i]]["label"]
    probs = zero_shot_classification(
        text,
        task_description,
        label_sentences,
    )

    probs_tensor = torch.tensor(probs)
    probs_softmax = F.softmax(probs_tensor, dim=0)

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1
        tqdm.write(f"Index {i}: {Fore.GREEN}Success 🎉 {Style.RESET_ALL} ")
        print(f"dataset index: {random_numbers[i]}")
        print(f"Confidence: {get_confidence(probs_softmax):.2f}")
        print(f"NLL: {calculate_nll(probs_softmax,label):.2f}")
    else:
        tqdm.write(f"Index {i}: {Fore.RED}Fail 🧐 {Style.RESET_ALL}")
        print(f"dataset index: {random_numbers[i]}")
        print(f"Confidence: {get_confidence(probs_softmax):.2f}")
        print(f"NLL: {calculate_nll(probs_softmax,label):.2f}")
        print(f"Failed Query: {Back.YELLOW}{text}{Back.RESET}")
        label, pred = label_map[label], label_map[pred]
        print(
            f"Label : {Fore.CYAN} {label} {Fore.RESET} Pred : {Fore.MAGENTA} {pred} {Fore.RESET}"
        )
    current_accuracy = n_corrects / (i + 1)
    print(f"Current Accuracy: {Fore.BLUE}{current_accuracy:,.2%}{Fore.RESET}")

print(f"{Fore.RED}>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>{Fore.RESET}")
print(
    f"Total Correct Predictions: {Style.BRIGHT}{n_corrects} ({n_corrects/total:.2%}){Style.RESET_ALL}"
)
print(f"{Fore.RED}>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>{Fore.RESET}")

  0%|          | 0/50 [00:00<?, ?it/s]

----------------------------------


  2%|▏         | 1/50 [00:04<03:32,  4.34s/it]

Index 0: Fail 🧐 
dataset index: 6254
Confidence: 21.58
NLL: 49.69
Failed Query: Powell Meets with Middle East Leaders CBN.com - (CBN News) -ERUSALEM  -h the death of Yasser Arafat behind them, the Palestinians and Israelis are now facing an historic opportunity to move forward in peace.
Label :  World  Pred :  Business 
Current Accuracy: 0.00%
----------------------------------


  4%|▍         | 2/50 [00:06<02:38,  3.30s/it]

...

Index 47: Fail 🧐 
dataset index: 5058
Confidence: 23.17
NLL: 82.03
Failed Query: Roma 1-1 Bayer Leverkusen: FT Report ROME, November 3 (Champions League) - Vincenzo Montella #39;s injury-time equaliser forced Bayer Leverkusen to settle for a share of the points on Wednesday in Group B of the Champions League but the Eternal City club are virtually eliminated if not yet 
Label :  Sports  Pred :  Business 
Current Accuracy: 22.92%
----------------------------------


 98%|█████████▊| 49/50 [02:30<00:02,  2.84s/it]

Index 48: Fail 🧐 
dataset index: 4456
Confidence: 16.99
NLL: 39.11
Failed Query: Victory Looms for Karzai as Vote Probe Continues Hamid Karzai was assured of a majority in Afghanistans election to become its first democratically chosen president. With nearly 95 per cent of votes counted, the interim leader already has more than half 
Label :  World  Pred :  Business 
Current Accuracy: 22.45%
----------------------------------


100%|██████████| 50/50 [02:33<00:00,  3.06s/it]

Index 49: Fail 🧐 
dataset index: 2414
Confidence: 27.29
NLL: inf
Failed Query: Samples From Genesis Craft Sent to Calif. (AP) AP - The first solar-wind samples recovered from the crashed Genesis space capsule have been sent to researchers in California.
Label :  Sci/Tech  Pred :  Business 
Current Accuracy: 22.00%
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Total Correct Predictions: 11 (22.00%)
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


---


# 실습: Zero-shot Classification

이번 실습에서는 open LLM을 가지고 zero-shot classification을 해봅니다. 먼저 필요한 library들을 설치합시다.


그 다음 Gemma-2B를 사용하기 위해 다음과 같은 작업을 진행합니다:

1. huggingface.co 계정 만들고 로그인하기
2. https://www.kaggle.com/models/google/gemma/license/consent 에서 Gemma license 동의하기
3. 홈 화면으로 돌아와, `Profile > Settings > Access Tokens` 메뉴로 들어와 "Write" type의 token 생성하기
4. 생성한 토큰을 아래 "HF TOKEN"에 불여넣고 셀을 실행하기.


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
HF_TOKEN = os.environ.get("HF_TOKEN")

from huggingface_hub import login

login(HF_TOKEN)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

이번에는 Gemma-2B를 가지고 간단한 text 생성을 해봅시다.
"What is your name?" 이라는 text를 넣었을 때 어떤 text가 생성되는지 살펴봅시다.


In [ ]:
input_text = "What is your name?"
input_ids = tokenizer(input_text, return_tensors="pt").to("mps")
input_ids

In [ ]:
input_text = "What is your name?"
input_ids = tokenizer(input_text, return_tensors="pt").to("mps")
outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
tokens = input_ids["input_ids"]
print(tokens)
input_ids

2B의 작은 LLM이라 질좋은 답변이 나오지 않는 것을 알 수 있습니다.
이번에는 입력으로 넣어준 token들의 logit을 계산해봅시다.


In [ ]:
tokens = input_ids["input_ids"]
print(tokens)

logits = model(**input_ids).logits
for i in range(tokens.shape[-1]):
    token = tokens[0, i].item()
    print(logits[0, i, token])

위와 같이 모델 출력의 `.logits`을 통해 token들의 logit을 알 수 있습니다.
Logit은 높을 수록 token이 나올 확률이 높다는 뜻입니다.

이번에는 logit 계산을 통해 zero-shot classification을 구현해보도록 하겠습니다.


In [8]:
import torch


def zero_shot_classification(
    text, task_description, labels
):  # text는 주어진 입력, task_description은 task에 대한 설명, labels은 class들을 text로 변환한 결과입니다.
    text_ids = tokenizer(task_description + text, return_tensors="pt").to(
        "mps"
    )  # 먼저 task_description과 text를 이어붙인 후, tokenize합니다.

    probs = []

    for label in labels:
        # 그 다음 각 text화된 label들을 tokenize하고 입력에 이어붙인 후, Gemma-2B에 넣어줍니다.
        label_ids = tokenizer(label, return_tensors="pt").to("mps")
        n_label_tokens = label_ids["input_ids"].shape[-1] - 1
        # text로 변환한 label의 token 수를 계산합니다.
        input_ids = {
            "input_ids": torch.concatenate(
                [text_ids["input_ids"], label_ids["input_ids"][:, 1:]], axis=-1
            ),  # concatenate 명령어를 통해 이어붙이는 모습입니다.
            "attention_mask": torch.concatenate(
                [text_ids["attention_mask"], label_ids["attention_mask"][:, 1:]],
                axis=-1,
            ),
        }

        logits = model(**input_ids).logits  # Logit을 계산한 모습입니다.
        prob = 0
        n_total = input_ids["input_ids"].shape[-1]
        for i in range(
            n_label_tokens, 0, -1
        ):  # 일반적으로 text로 변환한 label은 여러 token으로 이루어져있습니다. 이러한 label에 대한 logit은 구성하는 모든 token들의 logit들의 합으로 정의합니다.
            token = label_ids["input_ids"][0, i].item()
            prob += logits[0, n_total - i, token].item()
        probs.append(prob)

        del input_ids
        del logits
        torch.cuda.empty_cache()  # 위의 del과 empty_cache() 명령어를 통해 GPU를 제때 할당해제 해줍니다. 만약 GPU가 여유롭다면 지워주시는게 속도적으로 이득입니다.

    return probs

아래는 실제로 zero-shot classification을 해본 결과입니다.


In [ ]:
probs = zero_shot_classification(
    "I am happy!", "Is the sentence positive or negative?: ", ["positive", "negative"]
)
print(probs)

보시다시피 우리는 Gemma를 별도로 학습하지 않았음에도 불구하고 주어진 문장이 긍정적이라는 것을 정확하게 예측하고 있습니다.

다음은 영화 리뷰 감정 분석 task에 적용해봅시다.
먼저 data를 불러옵니다.


In [ ]:
from datasets import load_dataset


imdb = load_dataset("imdb")


def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=200, truncation=True)


tokenized_imdb = imdb.map(preprocess_function, batched=True)

그리고 `test` data에서 50개의 영화 리뷰에 대해 예측하는 코드는 다음과 같습니다.


In [ ]:
import numpy as np
from tqdm import tqdm


n_corrects = 0
for i in tqdm(range(50)):
    text = tokenized_imdb["test"][i]["text"]
    label = tokenized_imdb["test"][i]["label"]
    probs = zero_shot_classification(
        text,
        "A movie review is given. Decide that the movie review is positive or negative: ",
        labels=["Answer: negative.", "Answer: positive."],
    )

    pred = np.argmax(np.array(probs))
    if pred == label:
        n_corrects += 1

print(n_corrects)

In [ ]:
text = tokenized_imdb["test"][i]["text"]
text

In [ ]:
label = tokenized_imdb["test"][i]["label"]
label

보시다시피 정확도 88%로, 매우 높은 성능을 보이는 것을 알 수 있습니다.
